In [ ]:
import sys
sys.path.append('../../')
from utils.connection import postgres_connection
import pandas as pd

In [39]:
query = """
select * from healthcare.public.patients a 
"""

df = pd.read_sql(query, con=postgres_connection())

In [40]:
df.dtypes

patient_id             int64
name                  object
age                  float64
gender                object
registration_date     object
dtype: object

In [41]:
# transfomations
df['registration_date'] = pd.to_datetime(df['registration_date'], format='%Y-%m-%d')
df['age'] = df['age'].astype('Int64')

In [42]:
# Aggregations

df['age_group'] = pd.cut(df['age'], bins=[0, 18, 30, 50, 70, 100], labels=['0-18', '19-30', '31-50', '51-70', '71+'])

# Create a new column for the number of months since registration
df['months_since_registration'] = (pd.to_datetime('now') - df['registration_date']).dt.days // 30
df['months_since_registration'] = df['months_since_registration'].astype('Int64')

df['patient_type'] = pd.cut(df['months_since_registration'], bins=[0, 6, 24, 100 ], labels=['New', 'Regular', 'Old'])

In [43]:
df.head()

,patient_id,name,age,gender,registration_date,age_group,months_since_registration,patient_type
0,1,John Hill,89,Male,2023-08-23,71+,21,Regular
1,2,Adam Chavez,21,Male,2024-04-01,19-30,13,Regular
2,3,Troy Scott,36,Male,2020-03-04,31-50,63,Old
3,4,Joan Ross,77,Female,2021-09-24,71+,44,Old
4,5,Stephanie Giles,41,Female,2022-08-17,31-50,33,Old
